In [1]:
import os
import re
import math
import json

In [2]:
# Read data
model_name = 'CRI_C1C5'
exp_name = 'init'
eqn_path = os.path.join(os.getcwd(), '{}_{}.eqn'.format(model_name, exp_name))
with open(eqn_path, 'r') as f:
    raw_lines = f.read().splitlines()
# Create a list of dictionaries describing individual reactions: reactants, products, rate coefficient, comments (optional)
eqs = []
for line in raw_lines:
    if not line.startswith('#') and not line.startswith('{{'):
        if exp_name == 'init' or exp_name == 'emis':
            subline = re.search(r'.*\t\s(.*)\s:\s\t\s(.*)\s\t;', line).group(1,2)
            reac, prod = [re.findall(r'[A-Za-z]+[\w]*', s) for s in subline[0].split('=')]
            eqs.append(dict(reac=reac, prod=prod, coef=subline[1]))

#### Generic rate coefficients

In [3]:
# Read MCM generic rate coefficients
gen_eqn_path = os.path.join(os.getcwd(), 'mcm_generic_rate_coeffs.txt')
with open(gen_eqn_path, 'r') as f:
    gen_raw_lines = f.read().splitlines()
gen_eqs = []
for line in gen_raw_lines:
    name, expr = re.search(r'([A-Za-z]+[\w]*)\s=\s(.*)', line).group(1, 2)
    gen_eqs.append(dict(name=name, expr=expr))
# Store the names of generic rate coefficients in a list for later use
gen_cf_names = []
for eq in gen_eqs:
    gen_cf_names.append(eq['name'])

In [4]:
# Parameters
TEMP = 298 # temperature [K]
M = 2.46e+19;
N2 = 1.9434e+19;
O2 = 5.166e+18;
H2O = 2.7e+17;
# Dictionary with constant species values
const_spec = {'M': eval('M'), 'N2': eval('N2'), 'O2': eval('O2'), 'H2O': eval('H2O')}
# Calculate generic rate coefficients
KRO2NO = 2.7e-12*math.exp(360/TEMP)
KRO2HO2 = 2.91e-13*math.exp(1300/TEMP)
KAPHO2 = 5.2e-13*math.exp(980/TEMP)
KAPNO = 7.5e-12*math.exp(290/TEMP)
KRO2NO3 = 2.3e-12
KNO3AL = 1.4e-12*math.exp(-1860/TEMP)
KDEC = 1.00e+06
KROPRIM = 2.50e-14*math.exp(-300/TEMP)
KROSEC = 2.50e-14*math.exp(-300/TEMP)
KCH3O2 = 1.03e-13*math.exp(365/TEMP)
K298CH3O2 = 3.5e-13
K14ISOM1 = 3.00e+07*math.exp(-5300/TEMP)
KD0 = 1.10e-05*M*math.exp(-10100/TEMP)
KDI = 1.90e+17*math.exp(-14100/TEMP)
KRD = KD0/KDI
FCD = 0.30
NCD = 0.75-1.27*(math.log10(FCD))
FD = 10**(math.log10(FCD)/(1+(math.log10(KRD)/NCD)**2))
KBPAN = (KD0*KDI)*FD/(KD0+KDI)
KC0 = 3.28e-28*M*(TEMP/300)**(-6.87)
KCI = 1.125e-11*(TEMP/300)**(-1.105)
KRC = KC0/KCI
FCC = 0.30
NC = 0.75-1.27*(math.log10(FCC))
FC = 10**(math.log10(FCC)/(1+(math.log10(KRC)/NC)**2))
KFPAN = (KC0*KCI)*FC/(KC0+KCI)
K10 = 1.0e-31*M*(TEMP/300)**(-1.6)
K1I = 5.0e-11*(TEMP/300)**(-0.3)
KR1 = K10/K1I
FC1 = 0.85
NC1 = 0.75-1.27*(math.log10(FC1))
F1 = 10**(math.log10(FC1)/(1+(math.log10(KR1)/NC1)**2))
KMT01 = (K10*K1I)*F1/(K10+K1I)
K20 = 1.3e-31*M*(TEMP/300)**(-1.5)
K2I = 2.3e-11*(TEMP/300)**0.24
KR2 = K20/K2I
FC2 = 0.6
NC2 = 0.75-1.27*(math.log10(FC2))
F2 = 10**(math.log10(FC2)/(1+(math.log10(KR2)/NC2)**2))
KMT02 = (K20*K2I)*F2/(K20+K2I)
K30 = 3.6e-30*M*(TEMP/300)**(-4.1)
K3I = 1.9e-12*(TEMP/300)**0.2
KR3 = K30/K3I
FC3 = 0.35
NC3 = 0.75-1.27*(math.log10(FC3))
F3 = 10**(math.log10(FC3)/(1+(math.log10(KR3)/NC3)**2))
KMT03 = (K30*K3I)*F3/(K30+K3I)
K40 = 1.3e-3*M*(TEMP/300)**(-3.5)*math.exp(-11000/TEMP)
K4I = 9.7e+14*(TEMP/300)**0.1*math.exp(-11080/TEMP)
KR4 = K40/K4I
FC4 = 0.35
NC4 = 0.75-1.27*(math.log10(FC4))
F4 = 10**(math.log10(FC4)/(1+(math.log10(KR4)/NC4)**2))
KMT04 = (K40*K4I)*F4/(K40+K4I)
KMT05 = 1.44e-13*(1+(M/4.2e+19))
KMT06 = 1 + (1.40e-21*math.exp(2200/TEMP)*H2O)
K70 = 7.4e-31*M*(TEMP/300)**(-2.4)
K7I = 3.3e-11*(TEMP/300)**(-0.3)
KR7 = K70/K7I
FC7 = 0.81
NC7 = 0.75-1.27*(math.log10(FC7))
F7 = 10**(math.log10(FC7)/(1+(math.log10(KR7)/NC7)**2))
KMT07 = (K70*K7I)*F7/(K70+K7I)
K80 = 3.2e-30*M*(TEMP/300)**(-4.5)
K8I = 3.0e-11
KR8 = K80/K8I
FC8 = 0.41
NC8 = 0.75-1.27*(math.log10(FC8))
F8 = 10**(math.log10(FC8)/(1+(math.log10(KR8)/NC8)**2))
KMT08 = (K80*K8I)*F8/(K80+K8I)
K90 = 1.4e-31*M*(TEMP/300)**(-3.1)
K9I = 4.0e-12
KR9 = K90/K9I
FC9 = 0.4
NC9 = 0.75-1.27*(math.log10(FC9))
F9 = 10**(math.log10(FC9)/(1+(math.log10(KR9)/NC9)**2))
KMT09 = (K90*K9I)*F9/(K90+K9I)
K100 = 4.10e-05*M*math.exp(-10650/TEMP)
K10I = 6.0e+15*math.exp(-11170/TEMP)
KR10 = K100/K10I
FC10 = 0.4
NC10 = 0.75-1.27*(math.log10(FC10))
F10 = 10**(math.log10(FC10)/(1+(math.log10(KR10)/NC10)**2))
KMT10 = (K100*K10I)*F10/(K100+K10I)
K1 = 2.40e-14*math.exp(460/TEMP)
K3 = 6.50e-34*math.exp(1335/TEMP)
K4 = 2.70e-17*math.exp(2199/TEMP)
K2 = (K3*M)/(1+(K3*M/K4))
KMT11 = K1 + K2
K120 = 2.5e-31*M*(TEMP/300)**(-2.6)
K12I = 2.0e-12
KR12 = K120/K12I
FC12 = 0.53
NC12 = 0.75-1.27*(math.log10(FC12))
F12 = 10**(math.log10(FC12)/(1.0+(math.log10(KR12)/NC12)**2))
KMT12 = (K120*K12I*F12)/(K120+K12I)
K130 = 2.5e-30*M*(TEMP/300)**(-5.5)
K13I = 1.8e-11
KR13 = K130/K13I
FC13 = 0.36
NC13 = 0.75-1.27*(math.log10(FC13))
F13 = 10**(math.log10(FC13)/(1+(math.log10(KR13)/NC13)**2))
KMT13 = (K130*K13I)*F13/(K130+K13I)
K140 = 9.0e-5*math.exp(-9690/TEMP)*M
K14I = 1.1e+16*math.exp(-10560/TEMP)
KR14 = K140/K14I
FC14 = 0.36
NC14 = 0.75-1.27*(math.log10(FC14))
F14 = 10**(math.log10(FC14)/(1+(math.log10(KR14)/NC14)**2))
KMT14 = (K140*K14I)*F14/(K140+K14I)
K150 = 8.6e-29*M*(TEMP/300)**(-3.1)
K15I = 9.0e-12*(TEMP/300)**(-0.85)
KR15 = K150/K15I
FC15 = 0.48
NC15 = 0.75-1.27*(math.log10(FC15))
F15 = 10**(math.log10(FC15)/(1+(math.log10(KR15)/NC15)**2))
KMT15 = (K150*K15I)*F15/(K150+K15I)
K160 = 8e-27*M*(TEMP/300)**(-3.5)
K16I = 3.0e-11*(TEMP/300)**(-1)
KR16 = K160/K16I
FC16 = 0.5
NC16 = 0.75-1.27*(math.log10(FC16))
F16 = 10**(math.log10(FC16)/(1+(math.log10(KR16)/NC16)**2))
KMT16 = (K160*K16I)*F16/(K160+K16I)
K170 = 5.0e-30*M*(TEMP/300)**(-1.5)
K17I = 1.0e-12
KR17 = K170/K17I
FC17 = 0.17*math.exp(-51/TEMP)+math.exp(-TEMP/204)
NC17 = 0.75-1.27*(math.log10(FC17))
F17 = 10**(math.log10(FC17)/(1.0+(math.log10(KR17)/NC17)**2))
KMT17 = (K170*K17I*F17)/(K170+K17I)
KMT18 = 9.5e-39*O2*math.exp(5270/TEMP)/(1+7.5e-29*O2*math.exp(5610/TEMP))
KPPN0 = 1.7e-03*math.exp(-11280/TEMP)*M
KPPNI = 8.3e+16*math.exp(-13940/TEMP)
KRPPN = KPPN0/KPPNI
FCPPN = 0.36
NCPPN = 0.75-1.27*(math.log10(FCPPN))
FPPN = 10**(math.log10(FCPPN)/(1+(math.log10(KRPPN)/NCPPN)**2))
KBPPN = (KPPN0*KPPNI)*FCPPN/(KPPN0+KPPNI)

In [5]:
# Add generic rate coefficients to a dictionary
gen_eqs[0].setdefault('num', []).append(KRO2NO)
gen_eqs[1].setdefault('num', []).append(KRO2HO2)
gen_eqs[2].setdefault('num', []).append(KAPHO2)
gen_eqs[3].setdefault('num', []).append(KAPNO)
gen_eqs[4].setdefault('num', []).append(KRO2NO3)
gen_eqs[5].setdefault('num', []).append(KNO3AL)
gen_eqs[6].setdefault('num', []).append(KDEC)
gen_eqs[7].setdefault('num', []).append(KROPRIM)
gen_eqs[8].setdefault('num', []).append(KROSEC)
gen_eqs[9].setdefault('num', []).append(KCH3O2)
gen_eqs[10].setdefault('num', []).append(K298CH3O2)
gen_eqs[11].setdefault('num', []).append(K14ISOM1)
gen_eqs[12].setdefault('num', []).append(KD0)
gen_eqs[13].setdefault('num', []).append(KDI)
gen_eqs[14].setdefault('num', []).append(KRD)
gen_eqs[15].setdefault('num', []).append(FCD)
gen_eqs[16].setdefault('num', []).append(NCD)
gen_eqs[17].setdefault('num', []).append(FD)
gen_eqs[18].setdefault('num', []).append(KBPAN)
gen_eqs[19].setdefault('num', []).append(KC0)
gen_eqs[20].setdefault('num', []).append(KCI)
gen_eqs[21].setdefault('num', []).append(KRC)
gen_eqs[22].setdefault('num', []).append(FCC)
gen_eqs[23].setdefault('num', []).append(NC)
gen_eqs[24].setdefault('num', []).append(FC)
gen_eqs[25].setdefault('num', []).append(KFPAN)
gen_eqs[26].setdefault('num', []).append(K10)
gen_eqs[27].setdefault('num', []).append(K1I)
gen_eqs[28].setdefault('num', []).append(KR1)
gen_eqs[29].setdefault('num', []).append(FC1)
gen_eqs[30].setdefault('num', []).append(NC1)
gen_eqs[31].setdefault('num', []).append(F1)
gen_eqs[32].setdefault('num', []).append(KMT01)
gen_eqs[33].setdefault('num', []).append(K20)
gen_eqs[34].setdefault('num', []).append(K2I)
gen_eqs[35].setdefault('num', []).append(KR2)
gen_eqs[36].setdefault('num', []).append(FC2)
gen_eqs[37].setdefault('num', []).append(NC2)
gen_eqs[38].setdefault('num', []).append(F2)
gen_eqs[39].setdefault('num', []).append(KMT02)
gen_eqs[40].setdefault('num', []).append(K30)
gen_eqs[41].setdefault('num', []).append(K3I)
gen_eqs[42].setdefault('num', []).append(KR3)
gen_eqs[43].setdefault('num', []).append(FC3)
gen_eqs[44].setdefault('num', []).append(NC3)
gen_eqs[45].setdefault('num', []).append(F3)
gen_eqs[46].setdefault('num', []).append(KMT03)
gen_eqs[47].setdefault('num', []).append(K40)
gen_eqs[48].setdefault('num', []).append(K4I)
gen_eqs[49].setdefault('num', []).append(KR4)
gen_eqs[50].setdefault('num', []).append(FC4)
gen_eqs[51].setdefault('num', []).append(NC4)
gen_eqs[52].setdefault('num', []).append(F4)
gen_eqs[53].setdefault('num', []).append(KMT04)
gen_eqs[54].setdefault('num', []).append(KMT05)
gen_eqs[55].setdefault('num', []).append(KMT06)
gen_eqs[56].setdefault('num', []).append(K70)
gen_eqs[57].setdefault('num', []).append(K7I)
gen_eqs[58].setdefault('num', []).append(KR7)
gen_eqs[59].setdefault('num', []).append(FC7)
gen_eqs[60].setdefault('num', []).append(NC7)
gen_eqs[61].setdefault('num', []).append(F7)
gen_eqs[62].setdefault('num', []).append(KMT07)
gen_eqs[63].setdefault('num', []).append(K80)
gen_eqs[64].setdefault('num', []).append(K8I)
gen_eqs[65].setdefault('num', []).append(KR8)
gen_eqs[66].setdefault('num', []).append(FC8)
gen_eqs[67].setdefault('num', []).append(NC8)
gen_eqs[68].setdefault('num', []).append(F8)
gen_eqs[69].setdefault('num', []).append(KMT08)
gen_eqs[70].setdefault('num', []).append(K90)
gen_eqs[71].setdefault('num', []).append(K9I)
gen_eqs[72].setdefault('num', []).append(KR9)
gen_eqs[73].setdefault('num', []).append(FC9)
gen_eqs[74].setdefault('num', []).append(NC9)
gen_eqs[75].setdefault('num', []).append(F9)
gen_eqs[76].setdefault('num', []).append(KMT09)
gen_eqs[77].setdefault('num', []).append(K100)
gen_eqs[78].setdefault('num', []).append(K10I)
gen_eqs[79].setdefault('num', []).append(KR10)
gen_eqs[80].setdefault('num', []).append(FC10)
gen_eqs[81].setdefault('num', []).append(NC10)
gen_eqs[82].setdefault('num', []).append(F10)
gen_eqs[83].setdefault('num', []).append(KMT10)
gen_eqs[84].setdefault('num', []).append(K1)
gen_eqs[85].setdefault('num', []).append(K3)
gen_eqs[86].setdefault('num', []).append(K4)
gen_eqs[87].setdefault('num', []).append(K2)
gen_eqs[88].setdefault('num', []).append(KMT11)
gen_eqs[89].setdefault('num', []).append(K120)
gen_eqs[90].setdefault('num', []).append(K12I)
gen_eqs[91].setdefault('num', []).append(KR12)
gen_eqs[92].setdefault('num', []).append(FC12)
gen_eqs[93].setdefault('num', []).append(NC12)
gen_eqs[94].setdefault('num', []).append(F12)
gen_eqs[95].setdefault('num', []).append(KMT12)
gen_eqs[96].setdefault('num', []).append(K130)
gen_eqs[97].setdefault('num', []).append(K13I)
gen_eqs[98].setdefault('num', []).append(KR13)
gen_eqs[99].setdefault('num', []).append(FC13)
gen_eqs[100].setdefault('num', []).append(NC13)
gen_eqs[101].setdefault('num', []).append(F13)
gen_eqs[102].setdefault('num', []).append(KMT13)
gen_eqs[103].setdefault('num', []).append(K140)
gen_eqs[104].setdefault('num', []).append(K14I)
gen_eqs[105].setdefault('num', []).append(KR14)
gen_eqs[106].setdefault('num', []).append(FC14)
gen_eqs[107].setdefault('num', []).append(NC14)
gen_eqs[108].setdefault('num', []).append(F14)
gen_eqs[109].setdefault('num', []).append(KMT14)
gen_eqs[110].setdefault('num', []).append(K150)
gen_eqs[111].setdefault('num', []).append(K15I)
gen_eqs[112].setdefault('num', []).append(KR15)
gen_eqs[113].setdefault('num', []).append(FC15)
gen_eqs[114].setdefault('num', []).append(NC15)
gen_eqs[115].setdefault('num', []).append(F15)
gen_eqs[116].setdefault('num', []).append(KMT15)
gen_eqs[117].setdefault('num', []).append(K160)
gen_eqs[118].setdefault('num', []).append(K16I)
gen_eqs[119].setdefault('num', []).append(KR16)
gen_eqs[120].setdefault('num', []).append(FC16)
gen_eqs[121].setdefault('num', []).append(NC16)
gen_eqs[122].setdefault('num', []).append(F16)
gen_eqs[123].setdefault('num', []).append(KMT16)
gen_eqs[124].setdefault('num', []).append(K170)
gen_eqs[125].setdefault('num', []).append(K17I)
gen_eqs[126].setdefault('num', []).append(KR17)
gen_eqs[127].setdefault('num', []).append(FC17)
gen_eqs[128].setdefault('num', []).append(NC17)
gen_eqs[129].setdefault('num', []).append(F17)
gen_eqs[130].setdefault('num', []).append(KMT17)
gen_eqs[131].setdefault('num', []).append(KMT18)
gen_eqs[132].setdefault('num', []).append(KPPN0)
gen_eqs[133].setdefault('num', []).append(KPPNI)
gen_eqs[134].setdefault('num', []).append(KRPPN)
gen_eqs[135].setdefault('num', []).append(FCPPN)
gen_eqs[136].setdefault('num', []).append(NCPPN)
gen_eqs[137].setdefault('num', []).append(FPPN)
gen_eqs[138].setdefault('num', []).append(KBPPN)

In [6]:
# Add info about generic rate coefficients to reaction dictionaries
for geq in gen_eqs:
    for eq in eqs:
        if geq['name'] == eq['coef']:
            eq['num'] = geq['num'][0]

In [7]:
flt_re = r'-?[0-9]+\.*[0-9]*' # regular expression for float and integer number
for eq in eqs:
    if 'num' not in eq.keys():
        # Unique coefficients
        if eq['coef'] == '5.6D-34*N2*(TEMP/300)**(-2.6)*O2+6.0D-34*O2*(TEMP/300)**(-2.6)*O2':
            eq['num'] = 5.6e-34*N2*(TEMP/300)**(-2.6)*O2+6.0e-34*O2*(TEMP/300)**(-2.6)*O2
        elif eq['coef'] == '3.2D-11*EXP(67/TEMP)*O2+2.0D-11*EXP(130/TEMP)*N2':
            eq['num'] = 3.2e-11*math.exp(67/TEMP)*O2+2.0e-11*math.exp(130/TEMP)*N2
        elif eq['coef'] == '2.03D-16*(TEMP/300)**4.57*EXP(693/TEMP)':
            eq['num'] = 2.03e-16*(TEMP/300)**4.57*math.exp(693/TEMP)
        elif eq['coef'] == '2.20D-13*KMT06*EXP(600/TEMP)+1.90D-33*M*KMT06*EXP(980/TEMP)':
            eq['num'] = 2.20e-13*eval('KMT06')*math.exp(600/TEMP)+1.90e-33*M*eval('KMT06')*math.exp(980/TEMP)
        elif eq['coef'] == '1.20D-14*TEMP*EXP(287/TEMP)':
            eq['num'] = 1.20e-14*TEMP*math.exp(287/TEMP)
        # Others
        elif 'EXP' in eq['coef']:
            if eq['coef'].count('TEMP') == 1:
                if len(eq['coef'].split('*')) == 2:
                    # 8.0D-12*EXP(-2060/TEMP)
                    ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)\*EXP\(({flt_re})'.format(flt_re=flt_re), eq['coef'])[0]
                    p1 = float(re.sub(r'[DdEe]', r'e', ptn[0]))
                    p2 = float(ptn[1])
                    eq['num'] = p1 * math.exp(p2/TEMP)
                elif len(eq['coef'].split('*')) == 3:
                    if eq['coef'].split('*')[2] in const_spec.keys():
                        # 3.3D-39*EXP(530/TEMP)*O2
                        ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)\*EXP\(({flt_re}).*TEMP\)\*(\w+)'.format(flt_re=flt_re), eq['coef'])[0]
                        p1 = float(re.sub(r'[DdEe]', r'e', ptn[0]))
                        p2 = float(ptn[1])
                        eq['num'] = p1 * math.exp(p2/TEMP) * const_spec[ptn[2]]
                    elif eq['coef'].split('*')[2] not in const_spec.keys():
                        # 3.2D-13*EXP(690/TEMP)*1.0
                        ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)\*EXP\(({flt_re}).*\*({flt_re})'.format(flt_re=flt_re), eq['coef'])[0]
                        p1 = float(re.sub(r'[DdEe]', r'e', ptn[0]))
                        p2 = float(ptn[1])
                        p3 = float(eq['coef'].split('*')[2])
                        eq['num'] = p1 * math.exp(p2/TEMP) * p3
                    else:
                        print(eq)
                elif len(eq['coef'].split('*')) == 4:
                    # 1.82D-13*EXP(416/TEMP)*0.335*RO2
                    ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)\*EXP\(({flt_re}).*\*({flt_re})\*RO2$'.format(flt_re=flt_re), eq['coef'])[0]
                    p1 = float(re.sub(r'[DdEe]', r'e', ptn[0]))
                    p2 = float(ptn[1])
                    p3 = float(eq['coef'].split('*')[2])
                    eq['num'] = p1 * math.exp(p2/TEMP) * p3
                else:
                    print(eq)
            elif eq['coef'].count('TEMP') == 2:
                # 9.65D-20*TEMP**2.58*EXP(-1082/TEMP)
                if len(eq['coef'].split('*')) == 5:
                    ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)\*TEMP\*\*({flt_re})\*EXP\(({flt_re})\/TEMP\)$'.format(flt_re=flt_re), eq['coef'])[0]
                    p1 = float(re.sub(r'[DdEe]', r'e', ptn[0]))
                    p2 = float(ptn[1])
                    p3 = float(ptn[2])
                    eq['num'] = p1 * TEMP ** p2 * math.exp(p3/TEMP)
                else:
                    # 1.55D-17*TEMP**2*EXP(-61/TEMP)*0.736
                    ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)\*TEMP\*\*({flt_re})\*EXP\(({flt_re})\/TEMP\)\*({flt_re})$'.format(flt_re=flt_re), eq['coef'])[0]
                    p1 = float(re.sub(r'[DdEe]', r'e', ptn[0]))
                    p2, p3, p4 = map(float, ptn[1:])
                    eq['num'] = p1 * TEMP ** p2 * math.exp(p3/TEMP) * p4
            else:
                print(eq)
        elif 'J' in eq['coef']:
            if len(eq['coef'].split('*')) == 2:
                eq['num'] = float(eq['coef'].split('*')[1])
            else:
                eq['num'] = 1
        elif 'EXP' not in eq['coef']:
            if len(eq['coef'].split('*')) == 1:
                eq['num'] = float(re.sub(r'[DdEe]', r'e', eq['coef']))
            elif len(eq['coef'].split('*')) == 2:
                if 'RO2' in eq['coef']:
                    try:
                        # 6.70D-13*RO2
                        ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)\*RO2$'.format(flt_re=flt_re), eq['coef'])[0]
                        eq['num'] = float(re.sub(r'[DdEe]', r'e', ptn))
                    except IndexError:
                        # KRO2NO3*0.40
                        ptn = re.findall(r'(.*)\*({flt_re})$'.format(flt_re=flt_re), eq['coef'])[0]
                        p1 = eval(ptn[0])
                        p2 = float(ptn[1])
                        eq['num'] = p1 * p2
                elif eq['coef'].split('*')[1] in const_spec.keys():
                    # 2.14D-10*H2O
                    ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)\*(.*)$'.format(flt_re=flt_re), eq['coef'])[0]
                    p1 = float(re.sub(r'[DdEe]', r'e', ptn[0]))
                    p2 = eval(ptn[1])
                    eq['num'] = p1 * p2
                else:
                    try:
                        # 5.53D-12*0.51
                        ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)\*({flt_re})$'.format(flt_re=flt_re), eq['coef'])[0]
                        p1 = float(re.sub(r'[DdEe]', r'e', ptn[0]))
                        p2 = float(ptn[1])
                        eq['num'] = p1 * p2
                    except IndexError:
                        # KNO3AL*2.4
                        ptn = re.findall(r'(.*)\*({flt_re})$'.format(flt_re=flt_re), eq['coef'])[0]
                        p1 = eval(ptn[0])
                        p2 = float(ptn[1])
                        eq['num'] = p1 * p2
            elif len(eq['coef'].split('*')) == 3:
                if 'RO2' in eq['coef'].split('*')[1]:
                    # 2.50D-13*RO2*0.398
                    ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)\*RO2\*({flt_re})$'.format(flt_re=flt_re), eq['coef'])[0]
                    p1 = float(re.sub(r'[DdEe]', r'e', ptn[0]))
                    p2 = float(ptn[1])
                    eq['num'] = p1 * p2
                elif 'RO2' in eq['coef'].split('*')[2]:
                    ptn = re.findall(r'({flt_re}[DdEe] *[\+-?]*[0-9]+)\*({flt_re})\*RO2$'.format(flt_re=flt_re), eq['coef'])[0]
                    p1 = float(re.sub(r'[DdEe]', r'e', ptn[0]))
                    p2 = float(ptn[1])
                    eq['num'] = p1 * p2
                elif 'RO2' in eq['coef'].split('*')[0]:
                    # KRO2NO*0.917*0.602
                    ptn = re.findall(r'(.*)\*({flt_re})\*({flt_re})$'.format(flt_re=flt_re), eq['coef'])[0]
                    p1 = eval(ptn[0])
                    p2 = float(ptn[1])
                    p3 = float(ptn[2])
                    eq['num'] = p1 * p2 * p3
                else:
                    print(eq)
            else:
                print(eq)
        else:
            print(eq)

In [8]:
# Store reactions where rate coefficient potentially has not been calculated
unprocessed_reactions = [eq for eq in eqs if 'num' not in eq.keys()]
class UnprocessedReaction(Exception):
    pass
if len(unprocessed_reactions) != 0:
    raise UnprocessedReaction('Rate coefficient is not calculated.')

In [9]:
# Save reaction descriptions as a json file
eqs_json_path = os.getcwd()
if not os.path.exists(eqs_json_path):
    os.makedirs(eqs_json_path)
with open(os.path.join(eqs_json_path, '{}_{}.json'.format(model_name, exp_name)), 'w') as f:
    json.dump(eqs, f)